In [1]:
%pip install fooddatacentral
%pip install pint
%pip install eep153_tools
%pip install gnupg
%pip install --upgrade gspread
%pip install --upgrade gspread_pandas



  Using cached fooddatacentral-1.0.10-py3-none-any.whl.metadata (1.9 kB)
Using cached fooddatacentral-1.0.10-py3-none-any.whl (6.8 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached Pint-0.24.4-py3-none-any.whl.metadata (8.5 kB)
  Using cached flexcache-0.3-py3-none-any.whl.metadata (7.0 kB)
  Using cached flexparser-0.4-py3-none-any.whl.metadata (18 kB)
Using cached Pint-0.24.4-py3-none-any.whl (302 kB)
Using cached flexcache-0.3-py3-none-any.whl (13 kB)
Using cached flexparser-0.4-py3-none-any.whl (27 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached eep153_tools-0.12.4-py2.py3-none-any.whl.metadata (363 bytes)
Using cached eep153_tools-0.12.4-py2.py3-none-any.whl (4.9 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached gnupg-2.3.1-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.
  Using cached gspread-6.1.4-py3-none-any.whl.metadata (11 kB)
Using ca